mne.io.read_raw_edf: Loads the file and read the data

raw.info 

In [7]:
# Before running the code i had to download ipykernel
import os
import mne
import sys
import matplotlib.pyplot as plt
from mne.preprocessing import ICA

print(sys.executable)
%matplotlib qt


c:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\.venv-gpu\Scripts\python.exe


2. Load & combine data by task for 1 subject

Subject 1 data was loaded & combined based on the task
imagined dataset : runs 4, 8, 12
actual movement dataset: runs 3, 7, 11

In [8]:

def process_and_save_subject_task(subject_id, task_name, runs, base_raw_path, base_output_path):
    """
    Loads raw runs for a subject/task, combines them, and saves the result.
    """
    print(f"--- Processing Subject {subject_id}, Task: {task_name} ---")

    # 1. Load and Combine the raw data
    subject_folder = f"S{subject_id:03d}"
    subject_folder_path = os.path.join(base_raw_path, subject_folder)
    
    raw_files = []
    for run_number in runs:
        file_name = f"{subject_folder}R{run_number:02d}.edf"
        file_path = os.path.join(subject_folder_path, file_name)
        raw = mne.io.read_raw_edf(file_path, preload=True, stim_channel='auto')
        raw_files.append(raw)
        
    raw_combined = mne.concatenate_raws(raw_files)

    # 2. Define the output path and save the file
    output_folder = os.path.join(base_output_path, subject_folder)
    os.makedirs(output_folder, exist_ok=True)
    
    output_filename = f"{subject_folder}_{task_name}_raw.fif"
    output_path = os.path.join(output_folder, output_filename)
    
    raw_combined.save(output_path, overwrite=True)
    print(f"✅ Saved combined data to: {output_path}\n")

# --- Configuration ---
base_raw_path = r"C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\raw"
base_output_path = r"C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\processed"

TASK_RUNS = {
    'imagined_movement': [4, 8, 12],
    'actual_movement': [3, 7, 11]
}

# --- Now, you can easily process multiple subjects in a loop ---
# Let's process the first 3 subjects as an example
for subject in range(1, 2):
    process_and_save_subject_task(
        subject_id=subject,
        task_name='imagined_movement',
        runs=TASK_RUNS['imagined_movement'],
        base_raw_path=base_raw_path,
        base_output_path=base_output_path
    )
    process_and_save_subject_task(
        subject_id=subject,
        task_name='actual_movement',
        runs=TASK_RUNS['actual_movement'],
        base_raw_path=base_raw_path,
        base_output_path=base_output_path
    )

--- Processing Subject 1, Task: imagined_movement ---
Extracting EDF parameters from C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\raw\S001\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\raw\S001\S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\raw\S001\S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Overwriting existing file.
Writing C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\processed\S001\S001_imagined_movement_raw.fif

Load & prepare the reorganised and concatonated data for understanding

In [9]:
subject_id = 1
task_name = 'imagined_movement'

base_output_path = r"C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\processed"
subject_folder = f"S{subject_id:03d}"
processed_file = os.path.join(base_output_path, subject_folder, f"{subject_folder}_{task_name}_raw.fif")

raw = mne.io.read_raw_fif(processed_file, preload=True)

print(raw.info)
print(f"Data Duration: {raw.times[-1]/60:.2f} minutes")


Opening raw data file C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\Datasets\processed\S001\S001_imagined_movement_raw.fif...
Isotrak not found
    Range : 0 ... 59999 =      0.000 ...   374.994 secs
Ready.
Reading 0 ... 59999  =      0.000 ...   374.994 secs...
<Info | 10 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 meas_id: 4 items (dict)
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
 subject_info: <subject_info | his_id: X, last_name: X, sex: 0>
>
Data Duration: 6.25 minutes


Clean channel names & set montage (togographical map of nodes)

In [ ]:
raw.rename_channels(lambda name: name.replace('.', '').strip().upper())

raw.set_channel_types({ch: 'eeg' for ch in raw.ch_names})

montage = mne.channels.make_standard_montage('standard_1005')
raw.set_montage(montage, on_missing='ignore')

raw.plot_sensors(show_names=True)


ValueError: DigMontage is only a subset of info. There are 12 channel positions not present in the DigMontage. The channels missing from the montage are:

['FCZ', 'CZ', 'CPZ', 'FP1', 'FPZ', 'FP2', 'AFZ', 'FZ', 'PZ', 'POZ', 'OZ', 'IZ'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.

Grab Plotting of all regions before filtering
1. Visualise raw EEG signal in time domain.
2. Visualise the power spectral density of the EEG signal (how signal power is distributed across different frequncy)

In [ ]:
print("Showing all channels together...")
raw.plot(
    duration=5,            # how many seconds per screen
    n_channels=len(raw.ch_names),  # display all channels
    scalings='auto',       # automatically scale signal amplitude
    title='All EEG Channels',
    butterfly=True         # overlay channels for better comparison
)

psd = raw.compute_psd(fmax=60)
psd.plot(picks="eeg", exclude = "bads", average = True, dB = True)
# Pick - picks what type of signal it has to plot; eeg signal
# exlude - do not include bad channels
# average -


In [ ]:
raw.filter(l_freq=1., h_freq=40.)

Plotting all of the channels together after using the filter
- the regular raw.plot is unable to view the difference clearly

psd compute is used to see a better comparison when the filter is applied 
- View signal power distribution across different frequencies (filtering affects this graph)

In [ ]:
print("Showing all channels together...")
raw.plot(
    duration=5,            # how many seconds per screen
    n_channels=len(raw.ch_names),  # display all channels
    scalings='auto',       # automatically scale signal amplitude
    title='All EEG Channels',
    butterfly=True         # overlay channels for better comparison
)

psd = raw.compute_psd(fmax=60)
psd.plot(picks = "eeg",exclude="bads", average= True, dB = True)

Graph plotting of separate regions

In [ ]:
# Plot a short segment to visualize
# This will open an interactive plot window.

for regions, channels in channel_subgroups.items():
    if not channels:
        continue
    print (f"showing regions: {regions}\n Channels shown: {len(channels)} channels: {channels}")
    raw.plot(picks = channels, duration= 5, n_channels=len(channels), scalings = "auto", title = f"{regions} Region EEG")


Preprocessing takes place
ICA process starts (Independant Component Analysis)

In [ ]:
ica = ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)

ica.plot_sources(raw)
ica.exclude = [0, 4, 11]

# IMPORTANT: Always apply the cleaning to a copy of the data
raw_cleaned = raw.copy()
ica.apply(raw_cleaned)

print("Displaying original data...")
raw.plot(title="Original Data")

print("\nDisplaying data after removing artifacts...")
raw_cleaned.plot(title="Cleaned Data")
